In [1]:
import os
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import codecs
from os import path
import csv
import networkx as nx
import numpy as np
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from utils import get_raw_data, dump_prediction

In [2]:
import unicodedata

In [3]:
train_data, test_data, y_train, G, train_hosts, tests_hosts  = get_raw_data()

100%|█████████████████████████████████████████████████████████████████████████████████| 560/560 [00:00<00:00, 560307.79it/s]


In [6]:
enc = list()
import encodings
for elt in tqdm(sorted(set(encodings.aliases.aliases.values()))):
    for sub in sorted(set(encodings.aliases.aliases.values())):
        try:
            t = train_data[0].encode(elt).decode(sub).split()[1]
            #enc.append((str(etl), str(sub), t))
            enc.append(t)
        except:
            pass

100%|██████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 105.30it/s]


In [7]:
enc

['â»',
 'â»',
 '9ЕдЩЕPУЦЙЩ@@@@@@@@@@N@ГИБЩгЩЕв@ФЕгЩЦЧЦУЕ@@@@@@@@@@N@ФЕгЩЦЧЦУЕ@ДЕ@ЩЦдЕХ@@@@@@@@@@N@БЗЗУЦ@Дд@ЧБив@ДЕ@ДЩЕдз@@@@@@@@@@N@БЗЗУЦ@ЕеЩЕдз@ЧЦЩгЕв@ДЕ@ХЦЩФБХДЙЕ@@@@@@@@@@N@БЗЗУЦ@вЕЙХЕPЕдЩЕ@@@@@@@@@@N@БЗЗУЦ@ДЕ@гЦдЩв@ЧУдв@@@@@\\@ЧЕЩЖЦЩФБХГЕ@ЙХДдвгЩЙЕУУЕ@@@@@@@@@@N@ЕзГЕУУЕХГЕ@ЦЧF┤ЩБгЙЦХХЕУУЕ@@@@@@@@@@N@ФБЙХгЕХБХГЕ@@@@@@@@@@N@ЧБГТБЗЙХЗ@@@@@@@@@@N@вдЧЧУи@ГИБЙХ@@@@@@@@@@N@двЙХЕ@Дд@ЖдгдЩ@@@@@\\@гБУЕХгв@@@@@\\@ЩЕУБгЙЦХв@ЧдВУЙШдЕв@@@@@\\@ЙХХЦеБгЙЦХ@@@@@@@@@@N@ВЙЦФF┤ДЙГБФЕХгв@@@@@@@@@@N@ЖF┤ДF┤ЩЕЩ@@@@@@@@@@N@ГЦУУБВЦЩЕЩ@@@@@@@@@@N@ЧЩЦФЦдеЦЙЩ@@@БДИF┤ЩЕЩ@FA@ЧЦУЕЧИБЩФБ@@@ЕвЧБГЕ@БДИF┤ЩЕХг@@@ЩЕГИЕЩГИЕЩ@дХ@БДИF┤ЩЕХг@@@СЕ@ДF┤ГЦдеЩЕ@УЕв@ФF┤гЙЕЩв@ДЕ@УБ@ЖЙУЙF╜ЩЕ@ЧИБЩФБГЕдгЙШдЕ@@@ЧЦУЕЧИБЩФБ@@@ФF┤ДЙБгИF╜ШдЕ@@@ЕвЧБГЕ@ЧЩЕввЕ@@@ЧЦУЕЧИБЩФБ@ЧЦУЕЧИБЩФБ@@@WЛл@ЩЕГИЕЩГИЕmmmmmmmmm@вдВФЙг@@@ЧЦУЕЧИБЩФБ@ЧЦУЕЧИБЩФБ@@@@@\\@ЧЦУЕЧИБЩФБ@@@@@@@@@@N@ШдЙ@вЦФФЕв`ХЦдв@o@@@@@@@@@@N@ЦЩЗБХЙЗЩБФФЕ@@@@@@@@@@N@УЕв@ГИЙЖЖЩЕв@ГУF┤в@@@@@@@@@@N@дХ@ЩF┤вЕБд@@@@@\\@гЕЩЩЙгЦЙЩЕв@@@@@@@@@@N@ЩF┤ЗЙЦХ@ГЕХгЩЕ`еБУ@ДЕ@УЦЙЩЕ@@@@@@@@@@N@ЩF┤ЗЙЦХ@ХЦЩФБХД

In [17]:
PUNCTUATION = str.maketrans(dict.fromkeys('!.?,:-+*', " "))
BAD_CHARACTERS = ('\n', '\r', '\xa0', '&nbsp;')
from gensim.parsing.preprocessing import preprocess_documents

def process_doc(doc):
    proc = list(filter(lambda x: len(x) > 3, train_data[0].split()))
    return proc

In [18]:
process_doc(train_data[0])

['#polepharma',
 'flux',
 'polepharma',
 'flux',
 'commentaires',
 'alternate',
 'alternate',
 'polepharma',
 'polepharma',
 'recherche_________',
 'submit',
 '(button)',
 'toggle',
 'navigation',
 'polepharma',
 'polepharma',
 'sommes-nous',
 'organigramme',
 'chiffres',
 'clã©s',
 'rã©seau',
 'territoires',
 'rã©gion',
 'centre-val',
 'loire',
 'rã©gion',
 'normandie',
 'conseil',
 'dã©partemental',
 'dâ\x80\x99eure&loir',
 'chartres',
 'metropole',
 'metropole',
 'rouen',
 'agglo',
 'pays',
 'dreux',
 'agglo',
 'evreux',
 'portes',
 'normandie',
 'agglo',
 'seine&eure',
 'agglo',
 'tours',
 'plus',
 'performance',
 'industrielle',
 'excellence',
 'opã©rationnelle',
 'maintenance',
 'packaging',
 'supply',
 'chain',
 'usine',
 'futur',
 'talents',
 'relations',
 'publiques',
 'innovation',
 'biomã©dicaments',
 'fã©dã©rer',
 'collaborer',
 'promouvoir',
 'adhã©rer',
 'polepharma',
 'espace',
 'adhã©rent',
 'rechercher',
 'adhã©rent',
 'dã©couvre',
 'mã©tiers',
 'filiã¨re',
 'pharmaceu

In [28]:
from ftfy import fix_encoding
preprocess_documents([fix_encoding(train_data[0])])

[['polepharma',
  'flux',
  'polepharma',
  'flux',
  'de',
  'commentair',
  'altern',
  'altern',
  'polepharma',
  'polepharma',
  'ï»¿',
  'recherch',
  'submit',
  'button',
  'toggl',
  'navig',
  'polepharma',
  'polepharma',
  'qui',
  'somm',
  'nou',
  'organigramm',
  'le',
  'chiffr',
  'clã©',
  'rã©seau',
  'territoir',
  'rã©gion',
  'centr',
  'val',
  'loir',
  'rã©gion',
  'normandi',
  'conseil',
  'dã©partement',
  'd’eur',
  'loir',
  'chartr',
  'metropol',
  'metropol',
  'rouen',
  'agglo',
  'pai',
  'dreux',
  'agglo',
  'evreux',
  'port',
  'normandi',
  'agglo',
  'sein',
  'eur',
  'agglo',
  'tour',
  'plu',
  'perform',
  'industriel',
  'excel',
  'opã©rationnel',
  'mainten',
  'packag',
  'suppli',
  'chain',
  'usin',
  'futur',
  'talent',
  'relat',
  'publiqu',
  'innov',
  'biomã©dica',
  'fã©dã©rer',
  'collabor',
  'promouvoir',
  'adhã©rer',
  'polepharma',
  'espac',
  'adhã©rent',
  'recherch',
  'adhã©rent',
  'dã©couvr',
  'le',
  'mã©tier

In [46]:
res = list()
for elt in set(encodings.aliases.aliases.values()):
    for elt2 in set(encodings.aliases.aliases.values()):
        try:
            res.append('rã©gion'.encode(elt).decode(elt2))
        except:
            pass

In [48]:
'région' in res

False

In [53]:
text = dict()
filenames = os.listdir('text/text')
for filename in tqdm(filenames):
    with codecs.open(path.join('text/text/', filename), encoding='utf-8') as f: 
        test = f.read().replace("\n", "").lower()
        print(test)
        break

  0%|                                                                                             | 0/28003 [00:00<?, ?it/s]

    #alternate alternate     * français     * english     * deutsch     * chinese   rechercher sur le site rechercher sur le si submit   lorraine inp universite de lorraine   enstib     * accueil          + présentation de l'école          + portes ouvertes          + plaquette de l’enstib          + contacts et annuaires          + environnement et ouverture sociale          + web tv     * étudiants          + formations          + carrières          + associations étudiantes          + vie étudiante          + conventions     * recherche          + les laboratoires et les centres de transferts          + la formation par la recherche     * entreprises          + relations industrielles          + transfert de technologie          + parrainage          + service emploi          + formation continue          + taxe d’apprentissage 2019   contenuécole nationale supérieure des technologies et industries du boisl'enstib en 2 minutes 40formations enstib : pour quels métiers ?     * nicolas